# Batch Effects Present in Average Gene Expression Values
Exploring the distribution of absolute log fold change values between pairwise average gene expression vectors stratified by covariates.<br>
Note that throughout the notebook, we use *'rankit'* and *'quantile normalization'* interchangeably. A brief overview can be found <a href = 'https://en.wikipedia.org/wiki/Quantile_normalization'>here</a>.

In [1]:
import json
import plotly.figure_factory as ff
import plotly.express as px
import pingouin as pg
import plotly.graph_objects as go
import numpy as np
np.random.seed(123)

## Marker + Housekeeping Genes

In [2]:
NUM_MIN_EXPRESSED_GENES_PER_CELL = 500
NUM_MIN_ULTRA_LOW_EXPRESSED_GENES = 2
CENSUS_VERSION = "2023-10-18"
ASSAYS = ['sci-RNA-seq', 'Seq-Well', 'Drop-seq', 'CEL-seq2', "10x 3\\' v1", "10x 5\\' v1", "10x 3\\' v2", "10x 5\\' v2", "10x 3\\' v3", "10x 3\\' transcription profiling", "10x 5\\' transcription profiling", "10x technology"]
HOUSEKEEPING_GENES = ['MALAT1', 'ACTB', 'GAPDH', 'UBC', 'SDHA', 'YWHAZ', 'PGK1', 'PPIA', 'RPL13A', 'RPLP0', 'B2M']
NUM_MARKER_GENES = 5

contents_cell_types = open('../data/data_tidy/overlapping_categories.json').read()
contents_marker_genes = open('../data/data_raw/hubmap_marker_genes.json').read()
tissue2cell_types = json.loads(contents_cell_types)
marker_genes = json.loads(contents_marker_genes)

cell_type_mapping = {
    'CL:4028006' : 'alveolar type 2 fibroblast cell',
    'CL:0000525' : 'syncytiotrophoblast cell',
    'CL:0000786' : 'plasma cell',
    'CL:0000895' : 'naive thymus-derived CD4-positive, alpha-beta T cell',
    'CL:0000909' : 'CD8-positive, alpha-beta memory T cell',
    'CL:0000071' : 'blood vessel endothelial cell',
    'CL:0000899' : 'T-helper 17 cell',
    'CL:0000084' : 'T cell',
    'CL:1001106' : 'kidney loop of Henle thick ascending limb epithelial cell'
}
mapping2cell_type = {v:k for k, v in cell_type_mapping.items()}

def get_marker_genes(tissue, cell_type):
    return [entry['symbol'] for entry in marker_genes[mapping2cell_type[cell_type]] if entry['tissue'] == tissue]

## Rankit Implementation

In [3]:
import cellxgene_census
import numpy as np
import numba as nb
from scipy import stats
import scipy 
import pandas as pd
import scanpy as sc

@nb.jit
def quantiles(max_rank: int, ranks: np.ndarray) -> np.ndarray:
    """
    :returns an array of n floats equally spaced from 0 to 1
    """
    return np.array([np.round((i - 0.5) / max_rank, 5) for i in ranks])


def rankit(Xraw: scipy.sparse.spmatrix, offset: float = 3.0) -> scipy.sparse.csr_matrix:
    """
    Row-wise normalizes values of a matrix using the rankit method. The target distribution is a normal distribution
    with variance of 1 and mean as set in `offset`
    https://en.wikipedia.org/wiki/Rankit
    In statistics, rankits of a set of data are the expected values of the order statistics of
    a sample from the standard normal distribution the same size as the data
    Caveat: equal values are ranked in undefined order.
    param Xraw: query matrix to be normalized
    param offset: mean for the resulting row-wise values that will follow a normal distribution with variance 1. This
    helps to shift values to a positive scale.
    :returns row-wise normalized matrix using rankit
    """
    X = Xraw.tocsr(copy=True)  # get Compressed Sparse Row format of raw expression values matrix
    indptr = X.indptr  # get row count
    warning_raised = False
    for row in range(0, indptr.shape[0] - 1):
        data = X.data[indptr[row] : indptr[row + 1]]
        if len(data) > 0:
            # Assign ranks to data, assigning the same value to ties
            ranks = stats.rankdata(data, method="dense")

            max_rank = max(ranks)
            prob_level = quantiles(max_rank, ranks)

            normal_quantiles = stats.norm.ppf(prob_level, loc=offset)
            X.data[indptr[row] : indptr[row + 1]] = normal_quantiles
        elif not warning_raised:
            print("This dataset has at least one row of all zero expressions")
            warning_raised = True
    return X

## Analyses

In [4]:
def get_census_query(cell_types, tissues, assays):
    """
    Returns a value filter query for the census to retrieve data for 
    a given array of cell types, tissues and assays

    :param cell_types: the cell_types to retrieve from the census
    :param tissues: the tissues the cell_types are found in
    :param assays: the assays corresponding to the cell_types and tissues

    :return: the value_filter query
    """
    value_filter = "is_primary_data == True and cell_type in [" 
    for cell_type in cell_types:
        value_filter += "'" + cell_type.replace("'", "\\'") + "', "
    value_filter = value_filter[:-2] + "] and tissue_general in ["
    for tissue in tissues:
        value_filter += "'" + tissue + "', "
    value_filter = value_filter[:-2] + "] and assay in ["
    for assay in assays:
        value_filter += "'" + assay + "', "
    value_filter = value_filter[:-2] + "]"
    return value_filter

In [5]:
def get_census_metadata(cell_types, tissues, assays, census):
    """
    Queries the census to retrieve metadata for a given array of cell_types, tissues and assays

    :param cell_types: the cell_types to retrieve metadata for
    :param tissues: the tissues the cell_types are found in
    :param assays: the assays corresponding to the cell_types and tissues

    :return:
        metadata: metadata retrieved from the census 
        census_query: the query used to retrieve the metadata from cellxgene_census
        genes: the set of corresponding genes from the census
    """
    census_query = get_census_query(cell_types, tissues, assays)
    
    # Reads SOMADataFrame as a slice
    metadata = census["census_data"]["homo_sapiens"].obs.read(
        value_filter = census_query,
        column_names = ["soma_joinid", "assay", "dataset_id", "cell_type", "tissue", "tissue_general", "suspension_type", "disease", "donor_id", "raw_sum"]
    )

    genes = census["census_data"]["homo_sapiens"].ms["RNA"].var.read().concat().to_pandas()
        
    # Concatenates results to pyarrow.Table
    metadata = metadata.concat()
    
    # Converts to pandas.DataFrame
    metadata = metadata.to_pandas()
    # metadata['cell_type'] = metadata['cell_type'].apply(lambda x: cell_type2parents[x])
        
    print('There are', len(metadata), 'observations')
    return metadata, census_query, genes

### Get X iteratively and perform normalizations
**1. Pre-processing**
- *Remove cells with <500 expressed genes*

**2. Normalizations**
1. Rankit
2. log CPM

**3. Post-processing** <br>
- *Removal of Noisy Ultra-low Expression Values* <br>
After applying normalization, any gene/cell combination that had counts less or equal than 2 are set to missing data. This allows for removal of noise due to ultra-lowly expressed genes and provides a cleaner visualization.


In [6]:
from cellxgene_census.experimental.util import X_sparse_iter
import tiledbsoma as soma

def get_raw_count_data_and_normalize(cell_type, tissue, assays):
    """
    Retrieves raw counts data from the census for a given cell_type, tissue and set of assays 
    and performs quantile normalization and log (CPM) normalizations. Data is retrieved in batches.

    :param cell_type: cell_type to get data for
    :param tissue: corresponding tissue for the cell_type
    :param assays: assays corresponding to the cell_type and tissue

    :return:
        all_X_raw_counts: raw counts data
        all_X_rankit: quantile normalized data
        all_X_log_cpm: log (CPM) normalized data
        genes: the set of corresponding genes from the census
    """
    with cellxgene_census.open_soma(census_version=CENSUS_VERSION) as census:
        tissues = [tissue]
        cell_types = [cell_type]

        metadata, value_filter, genes = get_census_metadata(cell_types, tissues, assays, census)
        exp = census["census_data"]["homo_sapiens"]
        query = exp.axis_query(
        measurement_name = "RNA",
        obs_query = soma.AxisQuery(
            value_filter = value_filter
        ))
        all_obs_soma_joinids = []
        all_var_soma_joinids = []
        all_X_raw_counts = None
        all_X_rankit = None
        all_X_log_cpm = None

        i = 0
        for (obs_soma_joinids, var_soma_joinids), X_chunk in X_sparse_iter(query, X_name="raw", stride=10000):
            print('Parsing chunk Batch:', i)

            # Remove cells with < 500 expressed genes
            n_obs = X_chunk.shape[0]
            non_zero = [X_chunk[i,:].count_nonzero() for i in range(n_obs)]
            mask = [i >= NUM_MIN_EXPRESSED_GENES_PER_CELL for i in non_zero]
            obs_soma_joinids = obs_soma_joinids[mask]

            all_obs_soma_joinids.extend(obs_soma_joinids)
            all_var_soma_joinids.extend(var_soma_joinids)

            X_chunk = X_chunk[mask, :]

            print('\tCompute rankit values Batch:', i)
            # Compute rankit values
            X_rankit = rankit(X_chunk)

            # Removal of ultra-low expressed genes
            nonzero_mask = X_chunk.nonzero()
            X_chunk_nonzero = X_chunk[nonzero_mask]
            lowly_expressed_mask = np.array(X_chunk_nonzero <= NUM_MIN_ULTRA_LOW_EXPRESSED_GENES)[0]

            nonzero_rows_indices = nonzero_mask[0][lowly_expressed_mask]
            nonzero_cols_indices = nonzero_mask[1][lowly_expressed_mask]

            X_rankit[nonzero_rows_indices, nonzero_cols_indices] = 0
            X_rankit.eliminate_zeros()

            X_chunk[nonzero_rows_indices, nonzero_cols_indices] = 0
            X_chunk.eliminate_zeros()

            # Compute log CPM values
            print('\tCompute log CPM values Batch:', i)
            library_sizes = np.array(metadata[metadata['soma_joinid'].isin(obs_soma_joinids)]['raw_sum']).reshape(-1, 1)
            X_cpm = X_chunk / library_sizes * 1e6
            X_log_cpm = scipy.sparse.csr_matrix(np.log(X_cpm + 1))

            X_log_cpm[nonzero_rows_indices, nonzero_cols_indices] = 0
            X_log_cpm.eliminate_zeros()

            if i == 0:
                all_X_raw_counts = X_chunk
                all_X_rankit = X_rankit    
                all_X_log_cpm = X_log_cpm   
            else:
                all_X_raw_counts = scipy.sparse.vstack((all_X_raw_counts, X_chunk))
                all_X_rankit = scipy.sparse.vstack((all_X_rankit, X_rankit))
                all_X_log_cpm = scipy.sparse.vstack((all_X_log_cpm, X_log_cpm))
            i += 1
            # X_chunk is a scipy.csr_matrix of csc_matrix
            # For each X_chunk[i, j], the associated soma_joinid is
            # obs_soma_joinids[i] and var_soma_joinids[j]
    metadata = metadata[metadata['soma_joinid'].isin(all_obs_soma_joinids)]
    metadata['m_idx'] = [i for i in range(len(metadata))]

    obs_soma_joinid2idx = {x : i for i, x in enumerate(all_obs_soma_joinids)}
    obs_idx2soma_joinid = {v:k for k, v in obs_soma_joinid2idx.items()}

    var_idx2soma_joinid = {i : x for i, x in enumerate(all_var_soma_joinids)}
    var_soma_joinid2idx = {v:k for k, v in var_idx2soma_joinid.items()}
    return all_X_raw_counts, all_X_rankit, all_X_log_cpm, metadata, genes, obs_soma_joinid2idx, var_idx2soma_joinid

In [7]:
def get_average_ge(X):
    """
    Get average gene expression values for raw or normalized matrix X

    :param X: raw or normalized (quantile, log CPM) matrix of gene expression values

    :return: array containing average gene expression values, where each entry corresponds to the average for a specicic gene
    """
    covariate_gene_expression_sums = X.sum(axis = 0).tolist()[0]
    num_genes = X.shape[1]
    non_zero_cols, non_zero_counts = np.unique(X.indices, return_counts=True)
    averages = np.zeros(num_genes)
    for non_zero_col, non_zero_counts in zip(non_zero_cols, non_zero_counts):
        averages[non_zero_col] = covariate_gene_expression_sums[non_zero_col] / non_zero_counts
    return averages

In [8]:
from scipy.stats import f_oneway

def get_covariate_and_log2_fold_df(metadata, genes, X_raw, X_rankit, X_log_cpm, obs_soma_joinid2idx, covariate, genes_of_interest):
    """
    Computes average gene expression values per covariate and log2fold changes between pairwise average gene expression vectors

    :param metadata: dataframe containing metadata information about the cell type considered
    :param genes: the set of corresponding genes from the census
    :param X_raw: raw counts data
    :param X_rankit: quantile normalized data
    :param X_log_cpm: log (CPM) normalized data
    :param obs_soma_joinid2idx: mapping from obs_soma_join_id to idx in the X matrices
    :param covariate: covariate to consider (dataset_id or assay)
    :param genes_of_interest: genes to subset from the census

    :returns:
        covariate_raw_df: average gene expression values per covariate for raw counts
        covariate_rankit_df: average gene expression values per covariate for quantile normalized (rankit) data
        covariate_log_cpm_df: average gene expression values per covariate for log CPM normalized data
        covariate_df: average gene expression values per covariate aggregated over raw, quantile normalized and log CPM data
        covariate_log2_fc: dataframe containing pairwise log2 fold changes
    """
    covariates_grouped = metadata.groupby(covariate).aggregate(list)
    all_covariate_indices = covariates_grouped['soma_joinid'].to_list()
    num_covariates = len(all_covariate_indices)
    gene_expression_averages_raw = []
    gene_expression_averages_rankit = []
    gene_expression_averages_log_cpm = []

    gene_expression_raw_by_covariate = []
    gene_expression_rankit_by_covariate = []
    gene_expression_log_cpm_by_covariate = []
    print('There are', num_covariates, covariate)

    all_indices = []
    for covariate_idx in range(num_covariates):
        soma_join_ids = all_covariate_indices[covariate_idx]
        covariate_indices = [obs_soma_joinid2idx[x] for x in soma_join_ids]
        all_indices.extend(covariate_indices)

        covariate_gene_expression_avg_raw = get_average_ge(X_raw[covariate_indices])
        covariate_gene_expression_avg_rankit = get_average_ge(X_rankit[covariate_indices])
        covariate_gene_expression_avg_log_cpm = get_average_ge(X_log_cpm[covariate_indices])

        gene_expression_averages_raw.append(covariate_gene_expression_avg_raw)
        gene_expression_averages_rankit.append(covariate_gene_expression_avg_rankit)
        gene_expression_averages_log_cpm.append(covariate_gene_expression_avg_log_cpm)

        covariate_gene_expression_raw = X_raw[covariate_indices].toarray()
        covariate_gene_expression_rankit = X_rankit[covariate_indices].toarray()
        covariate_gene_expression_log_cpm = X_log_cpm[covariate_indices].toarray()

        gene_expression_raw_by_covariate.append(covariate_gene_expression_raw)
        gene_expression_rankit_by_covariate.append(covariate_gene_expression_rankit)
        gene_expression_log_cpm_by_covariate.append(covariate_gene_expression_log_cpm)

    covariates_list = covariates_grouped.index.to_list()
    covariate_df = pd.DataFrame(columns = ['cell_type_raw_count_avg', 'cell_type_rankit_avg', 'cell_type_log_cpm_avg'] + [x + '_raw_count_avg' for i, x in enumerate(covariates_list)] + [x +  '_rankit_avg' for i, x in enumerate(covariates_list)] + [x +  '_log_cpm_avg' for i, x in enumerate(covariates_list)])

    covariate_df['cell_type_raw_count_avg'] = get_average_ge(X_raw[all_indices])
    covariate_df['cell_type_rankit_avg'] = get_average_ge(X_rankit[all_indices])
    covariate_df['cell_type_log_cpm_avg'] = get_average_ge(X_log_cpm[all_indices])

    for cov_raw, cov_rankit, cov_log_cpm, cov in zip(gene_expression_averages_raw, gene_expression_averages_rankit, gene_expression_averages_log_cpm, covariates_list):
        covariate_df[cov + '_raw_count_avg'] = cov_raw
        covariate_df[cov + '_rankit_avg'] = cov_rankit
        covariate_df[cov + '_log_cpm_avg'] = cov_log_cpm

    covariate_df.index = genes_of_interest

    covariate_raw_df = covariate_df[[x for x in covariate_df.columns.to_list() if 'raw_count' in x]]
    covariate_rankit_df = covariate_df[[x for x in covariate_df.columns.to_list() if 'rankit' in x]]
    covariate_log_cpm_df = covariate_df[[x for x in covariate_df.columns.to_list() if 'log_cpm' in x]]

    covariate_raw_df_log2_fc, num_pairs, cov_raw_ttest = log2_fold_change(covariate_raw_df, gene_expression_raw_by_covariate, 'raw_counts')
    covariate_rankit_df_log2_fc, _, cov_rankit_ttest = log2_fold_change(covariate_rankit_df, gene_expression_rankit_by_covariate, 'rankit')
    covariate_log_cpm_df_log2_fc, _, cov_log_cpm_ttest = log2_fold_change(covariate_log_cpm_df, gene_expression_log_cpm_by_covariate, 'log CPM')

    covariates_grouped = metadata.groupby(covariate).aggregate(list)
    all_covariate_indices = covariates_grouped['soma_joinid'].to_list()
    num_genes = len(genes)

    covariate_raw_df_log2_fc['method'] = 'raw counts'
    covariate_rankit_df_log2_fc['method'] = 'rankit'
    covariate_log_cpm_df_log2_fc['method'] = 'log CPM'

    covariate_raw_df_log2_fc_new = get_log2_fold_stats_df(covariate_raw_df_log2_fc, cov_raw_ttest, 'raw counts')
    covariate_rankit_df_log2_fc_new = get_log2_fold_stats_df(covariate_rankit_df_log2_fc, cov_rankit_ttest, 'rankit')
    covariate_log_cpm_df_log2_fc_new = get_log2_fold_stats_df(covariate_log_cpm_df_log2_fc, cov_log_cpm_ttest, 'log CPM')

    covariate_log2_fc = pd.concat([covariate_raw_df_log2_fc_new, covariate_rankit_df_log2_fc_new, covariate_log_cpm_df_log2_fc_new], axis = 0)
    covariate_log2_fc.replace([np.inf, -np.inf], np.nan, inplace=True)
    covariate_log2_fc.dropna(inplace=True)
    covariate_log2_fc['covariate'] = covariate

    return covariate_raw_df, covariate_rankit_df, covariate_log_cpm_df, covariate_df, covariate_log2_fc, cov_raw_ttest

In [9]:
from scipy import stats
def log2_fold_change(covariate_df, gene_expressions, method):
    """
    Computes log2_fold_changes between pairs of vectors from covariate_df

    :param covariate_df: dataframe containing vectors to compute log2_fold_changes for
    :param gene_expressions: vector containing gene_expression values (raw or normalized)
    :param method: method used (raw counts, rankit, or log CPM)

    :return:
        new_cov_df: dataframe containing log2_fold changes
        num_pairs: number of pairs considered
        new_ttest_df: dataframe containing ttest p-values for the pairs considered
    """
    columns = covariate_df.columns[1:]
    num_cols = len(columns)
    num_pairs = num_cols * (num_cols - 1) / 2
    pair2log_fold = {}
    pair2mean = {}
    pair2ttest = {}
    col_names = []
    means = []
    for i, c1 in enumerate(columns):
        for j, c2 in enumerate(columns):
            col_name = 'P_' + str(i) + '_' + str(j)
            ttest = stats.ttest_ind(gene_expressions[i], gene_expressions[j], equal_var=False)
            statistic = ttest.statistic
            pvalues = ttest.pvalue
            if j > i:
                pair2log_fold[col_name] = np.log2(covariate_df[c1]) - np.log2(covariate_df[c2])
                pair2mean[col_name] = covariate_df[c1]
                col_names.append(col_name)
                pair2ttest[col_name] = (statistic, pvalues)
    new_covariate_df = pd.DataFrame(columns = col_names)
    ttest_df = pd.DataFrame()
    for col in col_names:
        new_covariate_df[col] = pair2log_fold[col]
        new_covariate_df[col + "_avg"] = pair2mean[col]
        ttest_df[col + "_statistic"] = pair2ttest[col][0]
        ttest_df[col + "_p_value"] = pair2ttest[col][1]
    new_cov_df = new_covariate_df.T
    new_ttest_df = ttest_df.T
    new_ttest_df.columns = new_cov_df.columns
    new_ttest_df['method'] = method
    new_cov_df['method'] = method
    return new_cov_df, num_pairs, new_ttest_df

### Log Fold Distributions

In [10]:
def get_log2_fold_stats_df(method_df, ttest_df, method):
    """
    Returns a dataframe containing log2 fold changes and statistics, such as mean, ttest statistic and p_value for each 
    pair of gene expression vectors

    :param method_df: dataframe containing log2 fold changes for the given method
    :param ttest_df: dataframe containing ttest statistics for the given method
    :param method: one of raw counts, rankit of log (CPM)

    :returns: dataframe containing pairwise log2 fold change, mean, gene_name, ttest statistic, ttest p_value and method
    """
    columns = method_df.columns[:-1]
    rows_mean = [x for x in list(method_df.index) if 'avg' in x]
    rows_dist = [x for x in list(method_df.index) if 'avg' not in x]

    rows_ttest = [x for x in list(ttest_df.index) if 'statistic' in x]
    rows_p_value = [x for x in list(ttest_df.index) if 'p_value' in x]

    pairwise_log_fold_changes = []
    ttest_stats = []
    ttest_pvalues = []
    gene_names = []
    methods = []
    means = []
    for col in columns:
        gene_expression_col = method_df.loc[rows_dist, col].to_list()
        mean_col = method_df.loc[rows_mean, col].to_list()
        ttest_stats_col = ttest_df.loc[rows_ttest, col].to_list()
        ttest_pval_col = ttest_df.loc[rows_p_value, col].to_list()

        pairwise_log_fold_changes.extend(gene_expression_col)
        means.extend(mean_col)
        ttest_stats.extend(ttest_stats_col)
        ttest_pvalues.extend(ttest_pval_col)
        gene_names.extend([col] * len(gene_expression_col))
        methods.extend([method] * len(gene_expression_col))
    df = pd.DataFrame({'pairwise log2 fold change' : pairwise_log_fold_changes, 'mean' : means, 'gene_name' : gene_names, 'ttest statistic' : ttest_stats, 'ttest p_value' : ttest_pvalues, 'method' : methods})
    return df

In [11]:
def transform_covariate_df(covariate_df, method_type, covariate, cell, tissue):
    """
    Transposes and augments a given dataframe by addint method, covariate and cell_type as new fields

    :param covariate_df: the df to transform
    :param method_type: one of raw_counts, rankit or log (CPM)
    :param covariate: type of covariate - usually dataset_id or assay
    :param cell: cell_type for the corresponding covariate_df
    :param tissue: tissue for the corresponding covariate_df
    """
    covariate_df = covariate_df.T
    covariate_df['method'] = method_type
    covariate_df['covariate'] = covariate
    covariate_df['cell_type'] = cell + ", " + tissue
    return covariate_df

## Choose cell types

In [12]:
# code used to generate cell types as leaf nodes in the ontology
cell_types_set = True
if not cell_types_set:
    with cellxgene_census.open_soma(census_version=CENSUS_VERSION) as census:
        human = census["census_data"]["homo_sapiens"]
        obs_df = human.obs.read(column_names=["cell_type_ontology_term_id", "cell_type", "tissue", "tissue_general"]).concat().to_pandas()
        obs_df.groupby(by=["cell_type_ontology_term_id", "cell_type"], as_index=False, observed=True).size()

        obs_df_unique = obs_df[['cell_type', 'cell_type_ontology_term_id']].drop_duplicates()
        cell_type_id2cell_type = {}
        for idx, row in obs_df_unique.iterrows():
            cell_type_id2cell_type[row['cell_type_ontology_term_id']] = row['cell_type']

        tissues_general = obs_df['tissue_general'].unique()

        # choose 5 random tissues that have cell types that are leaves in the ontology
        tissue2cell_types_leaves = json.loads(open('../data/terminal_cell_types.json').read())
        tissues_general_leaves = np.random.choice([x for x in tissue2cell_types_leaves.keys() if x in tissues_general], 5, replace = False)

        #  for each tissue, select 2 cell types that are leaves in the ontology at random
        cell_types = []
        for t in tissues_general_leaves:
            cell_type_leaves = tissue2cell_types_leaves[t]
            if len(cell_type_leaves) > 2:
                cell_type_leaves = np.random.choice(cell_type_leaves, 2, replace = False)
            for c in cell_type_leaves:
                cell_types.append((cell_type_id2cell_type[c], t))

## Generate log2 fold figure

In [13]:
def sample_obs_per_cell_type(cell_types, log2_fold_df, num_obs_per_cell_type = 30000):
    """
    Samples the log2_fold_df dataframe to contain a maximum of num_obs_per_cell_type observations
    per cell_type

    :param cell_types: the cell_types in log2_fold_df
    :param log2_fold_df: datafrane containing log2_fold pairwise changes
    :param num_obs_per_cell_type: number of max obs to sample to per cell_type

    :return: sampled log2_fold_df where each cell_type has a maximum of num_obs_per_cell_type observations
    """
    all_indices = []
    for ct in cell_types:
        cell_type_indices = log2_fold_df[log2_fold_df['cell_type'] == ct].index
        indices = np.random.choice(cell_type_indices, min(num_obs_per_cell_type, len(cell_type_indices)), replace = False)
        all_indices.extend(indices)
    log2_fold_df = log2_fold_df.iloc[all_indices]
    return log2_fold_df

In [14]:
def get_pairwise_stratified_log2_fold_changes(cell_types, covariates = ['dataset_id', 'assay'], all_genes = False):
    """
    Retrieve pairwise log2fold changes for a given set of [(cell_type, tissue)] pairs stratified by covariates
    Computes values for raw counts, quantile normalization and log (CPM)

    :param cell_types: array of [(cell_type, tissue)] pairs
    :param covariates: covariates to consider
    :param all_genes: True if to consider the entire gene expression vector; False if to consider only the marker genes

    :return: dataframe containing the pairwise log2fold changes for a given set of [(cell_type, tissue)] pairs stratified by covariates
    """
    all_dfs = []
    all_covariate_dfs_raw = []
    all_covariate_dfs_rankit = []
    all_covariate_dfs_log_cpm = []
    genes_set = False
        

    for cell_type, tissue in cell_types:

        X_raw_counts, X_rankit, X_log_cpm, metadata, genes, obs_soma_joinid2idx, var_idx2soma_joinid = get_raw_count_data_and_normalize(cell_type, tissue, ASSAYS)
        if len(metadata) == 0:
            continue
        if not genes_set:
            marker_genes = genes['feature_name'].to_list()
            genes_set = True
        var_soma_joinid2idx = {i : x for i, x in var_idx2soma_joinid.items()}

        if not all_genes:
            marker_genes = get_marker_genes(tissue, cell_type) 
        marker_genes_indices =  list([var_soma_joinid2idx[genes[genes['feature_name'] == x]['soma_joinid'].values[0]] for x in marker_genes])
        
        for genes_indices, genes_type, genes_of_interest in zip([marker_genes_indices], ['gene'], [marker_genes]):
            for covariate in covariates:
                covariate_raw_df, covariate_rankit_df, covariate_log_cpm_df, covariate_df, covariate_log2_fc, cov_raw_ttest = get_covariate_and_log2_fold_df(metadata, genes, X_raw_counts[:, genes_indices], X_rankit[:, genes_indices], X_log_cpm[:, genes_indices], obs_soma_joinid2idx, covariate, genes_of_interest)

                covariate_log2_fc['cell_type'] = cell_type + ", " + tissue
                if len(covariate_rankit_df) < 2:
                    continue

                covariate_log2_fc['gene_type'] = genes_type
            
                all_dfs.append(covariate_log2_fc)
                covariate_raw_df = transform_covariate_df(covariate_raw_df, 'raw', covariate, cell_type, tissue)
                covariate_rankit_df = transform_covariate_df(covariate_rankit_df, 'raw', covariate, cell_type, tissue)
                covariate_log_cpm_df = transform_covariate_df(covariate_log_cpm_df, 'raw', covariate, cell_type, tissue)

                all_covariate_dfs_raw.append(covariate_raw_df)
                all_covariate_dfs_rankit.append(covariate_rankit_df)
                all_covariate_dfs_log_cpm.append(covariate_log_cpm_df)
        
    log2_fold_df = pd.concat(all_dfs, axis= 0)
    return log2_fold_df, all_covariate_dfs_rankit

In [15]:
def generate_figure(log2_fold_df, num_assays_arr, num_datasets_arr, max_pts = 300000):
    """
    Generates plots of absolute log2fold change distribution between pairwise average gene expression values 
    stratified by covariates

    :param log2_fold_df: datafrane containing log2_fold pairwise changes 
    :param num_assays_arr: array containing number of assays per cell_type
    :param num_datasets_arr: array containing number of datasets per cell_type
    :returns: nothing
    """
    colors = ["#b19fc9", "#88d5d4", "#f7cb6c", "#619c80", "#df7670", "#f3a5ac"]
    log2_fold_df['method'] = log2_fold_df['method'].apply(lambda x: 'quantile normalization' if x == 'rankit' else x)
    
    if len(log2_fold_df) > max_pts:   
        log2_fold_df_sampled = log2_fold_df.sample(max_pts)
    else:
        log2_fold_df_sampled = log2_fold_df
    fig = px.box(log2_fold_df_sampled, y="pairwise log2 fold change", x = 'cell_type', facet_col = "covariate", color = "method", color_discrete_sequence = colors, category_orders={"method": ["raw counts", "quantile normalization", "log CPM"], "cell_type" : cell_types})

    fig.update_layout(width=2000,height=1000, title_text="<b>Absolute log2Fold change distribution between pairwise average gene expression<br> values stratified by covariates</b><br><br>", title_x=0.5, title_y = 0.98)

    fig.add_hline(y=1, line_width=2, line_dash="dash", line_color="gray", annotation_text = "log FC = 1")
    fig.update_traces(marker=dict(size=2))
    for x, num_datasets, num_assays in zip(sorted_cell_types, num_datasets_arr, num_assays_arr):
        fig.add_annotation(row=1,col=1, x=x, y=-0.3, text="N=" + str(int(num_assays)), arrowhead=False, showarrow = False)
        fig.add_annotation(row=1,col=2, x=x, y=-0.3, text="N=" + str(int(num_datasets)), arrowhead=False, showarrow=False)
    display(fig)

In [ ]:
# Note that these cells were chosen because they were leaves in the ontology. See the code that was used under 'Choose cell type' section above
cell_types = [('CD16-positive, CD56-dim natural killer cell, human', 'lung'), ('IgA plasma cell', 'small intestine'), ('central memory CD8-positive, alpha-beta T cell', 'blood'), ('enterocyte of colon', 'small intestine'), ('kidney loop of Henle thin ascending limb epithelial cell', 'kidney'), ('mucosal invariant T cell', 'blood'), ('naive thymus-derived CD8-positive, alpha-beta T cell', 'kidney'), ('plasmacytoid dendritic cell, human', 'blood'), ('type I pneumocyte', 'lung')]
log2_fold_df, all_covariate_dfs_rankit = get_pairwise_stratified_log2_fold_changes(cell_types, covariates = ['dataset_id', 'assay'], all_genes = True)
all_covariate_rankits = pd.concat(all_covariate_dfs_rankit)
for cell, tissue in cell_types:
    cell_name = cell + ", " + tissue
    cov_cell_df = all_covariate_rankits[all_covariate_rankits['cell_type'] == cell_name]

    num_datasets = len(cov_cell_df[cov_cell_df['covariate'] == 'dataset_id']) - 1
    num_assays = len(cov_cell_df[cov_cell_df['covariate'] == 'assay']) - 1

    log2_fold_df.loc[(log2_fold_df['cell_type'] == cell_name), 'num_datasets'] = num_datasets
    log2_fold_df.loc[(log2_fold_df['cell_type'] == cell_name), 'num_assays'] = num_assays

log2_fold_df = log2_fold_df[['pairwise log2 fold change', 'method', 'covariate', 'cell_type', 'gene_name', 'ttest p_value', 'num_datasets', 'num_assays', 'mean']]

# take absolute value
log2_fold_df['pairwise log2 fold change'] = log2_fold_df['pairwise log2 fold change'].apply(lambda x: abs(x))
log2_fold_df = log2_fold_df[(log2_fold_df['num_datasets'] > 1) & (log2_fold_df['num_assays'] > 1)]
sorted_cell_types = sorted(log2_fold_df['cell_type'].unique())
log2_fold_df = log2_fold_df.sort_values('cell_type').reset_index()
num_assays_arr = [log2_fold_df[log2_fold_df['cell_type'] == c]['num_assays'].values[0] for c in sorted_cell_types]
num_datasets_arr = [log2_fold_df[log2_fold_df['cell_type'] == c]['num_datasets'].values[0] for c in sorted_cell_types]


log2_fold_sampled_df = sample_obs_per_cell_type(sorted_cell_types, log2_fold_df)
generate_figure(log2_fold_sampled_df, num_assays_arr, num_datasets_arr)

There are 3458 observations
Parsing chunk Batch: 0
	Compute rankit values Batch: 0
	Compute log CPM values Batch: 0


/usr/local/anaconda3/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


There are 2 dataset_id


/usr/local/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


There are 3 assay


...

/usr/local/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_covariate_df[col + "_avg"] = pair2mean[col]
/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_covariate_df[col + "_avg"] = pair2mean[col

/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ttest_df[col + "_p_value"] = pair2ttest[col][1]
/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_covariate_df[col + "_avg"] = pair2mean[col]
/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.in

/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_covariate_df[col + "_avg"] = pair2mean[col]
/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ttest_df[col + "_statistic"] = pair2ttest[col][0]
/var/folders/5x/7vqbx7g95jl3tg9t0h1_fp400000gq/T/ipykernel_1405/1998459185.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

There are 6 assay


/usr/local/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Compute standard deviations

In [ ]:
log2_fold_df.head()

In [ ]:
import statistics
log2_fold_raw_counts_df = log2_fold_df[log2_fold_df['method'] == 'raw counts']
log2_fold_rankit_df = log2_fold_df[log2_fold_df['method'] == 'rankit']
log2_fold_log_cpm_df = log2_fold_df[log2_fold_df['method'] == 'ln (CPM + 1)']

stds = []
covariates = []
methods = []
cell_types_arr = []
cell_types = log2_fold_df['cell_type'].unique()

for cell_type in cell_types:
    for covariate in ['dataset_id', 'assay']:
        log2_fold_raw_counts_covariate_ct = log2_fold_raw_counts_df[(log2_fold_raw_counts_df['covariate'] == covariate) & (log2_fold_raw_counts_df['cell_type'] == cell_type)]
        log2_fold_rankit_covariate_ct = log2_fold_rankit_df[(log2_fold_rankit_df['covariate'] == covariate) & (log2_fold_rankit_df['cell_type'] == cell_type)]
        log2_fold_log_cpm_covariate_ct = log2_fold_log_cpm_df[(log2_fold_log_cpm_df['covariate'] == covariate) & (log2_fold_log_cpm_df['cell_type'] == cell_type)]

        n = len(log2_fold_raw_counts_covariate_ct)
        n_total = len(log2_fold_raw_counts_df[log2_fold_raw_counts_df['covariate'] == covariate])
        k = log2_fold_rankit_df['cell_type'].nunique()

        std_covariate_raw = statistics.variance(log2_fold_raw_counts_covariate_ct['pairwise log2 fold change'].values) * ((n - 1) / (n_total - k))
        std_covariate_rankit = statistics.variance(log2_fold_rankit_covariate_ct['pairwise log2 fold change'].values) * ((n - 1) / (n_total - k))
        std_covariate_log_cpm = statistics.variance(log2_fold_log_cpm_covariate_ct['pairwise log2 fold change'].values) * ((n - 1) / (n_total - k))

        stds.extend([std_covariate_raw, std_covariate_rankit, std_covariate_log_cpm])
        covariates.extend([covariate] * 3)
        cell_types_arr.extend([cell_type] * 3)
        methods.extend(['raw counts', 'rankit', 'log CPM'])
std_df = pd.DataFrame({'cell_type' : cell_types_arr, 'covariate' : covariates, 'method' : methods, 'std' : stds})

## Average variances

In [ ]:
np.sqrt(std_df.groupby(['method', 'covariate']).sum())